In [43]:
def zeros(*dims):
    return np.zeros(shape=tuple(dims), dtype=np.float32)

def ones(*dims):
    return np.ones(shape=tuple(dims), dtype=np.float32)

def rand(*dims):
    return np.random.rand(*dims).astype(np.float32)

def randn(*dims):
    return np.random.randn(*dims).astype(np.float32)

def sigmoid(batch, stochastic=False):
    return  1.0 / (1.0 + np.exp(-batch))

def as_matrix(vector):
    return np.reshape(vector, (-1, 1))

In [44]:
import nltk
import numpy as np
import pandas as pd
# import unicodecsv
# from collections import namedtuple

# response = []
# f= open('final_dataset.csv', 'rb')  
# reader = unicodecsv.DictReader(f)  

# for row in reader:  
#     response.append(row) 
# example = response[]
# f.close()


class StringConverter(dict):
    def __contains__(self, item):
        return True

    def __getitem__(self, item):
        return str

    def get(self, default=None):
        return str
pd.read_csv('final_dataset.csv', converters=StringConverter())
data = pd.read_csv('final_dataset.csv',encoding ='unicode escape')


In [45]:
train_dict = pd.Series(tokens).unique().tolist()
train_tokens = np.array([train_dict.index(token) for token in tokens])

In [46]:
train_set = []
for i in range(2,len(tokens)-2):
    train_set.append([train_dict.index(tokens[i]), train_dict.index(tokens[i-1])])
    train_set.append([train_dict.index(tokens[i]), train_dict.index(tokens[i-2])])
    train_set.append([train_dict.index(tokens[i]), train_dict.index(tokens[i+1])])
    train_set.append([train_dict.index(tokens[i]), train_dict.index(tokens[i+2])])

train_set = np.random.permutation(np.array(train_set))

In [47]:
Config = namedtuple("Config", ["dict_size", "vect_size", "neg_samples", "updates", "learning_rate",
                               "learning_rate_decay", "decay_period", "log_period"])
conf = Config(
    dict_size=len(train_dict),
    vect_size=100,
    neg_samples=10,
    updates=1000000,
    learning_rate=0.1,
    learning_rate_decay=0.995,
    decay_period=10000,
    log_period=10000)

In [48]:
def neg_sample(conf, train_set, train_tokens):
    Vp = randn(conf.dict_size, conf.vect_size)
    Vo = randn(conf.dict_size, conf.vect_size)

    J = 0.0
    learning_rate = conf.learning_rate
    for i in range(conf.updates):
        idx = i % len(train_set)

        word = train_set[idx, 0]
        context = train_set[idx, 1]

        neg_context = np.random.randint(0, len(train_tokens), conf.neg_samples)
        neg_context = train_tokens[neg_context]

        word_vect = Vp[word, :]  # word vector
        context_vect = Vo[context, :];  # context wector
        negative_vects = Vo[neg_context, :]  # sampled negative vectors

        # Cost and gradient calculation starts here
        score_pos = word_vect @ context_vect.T
        score_neg = word_vect @ negative_vects.T

        J -= np.log(sigmoid(score_pos)) + np.sum(np.log(sigmoid(-score_neg)))
        if (i + 1) % conf.log_period == 0:
            print('Update {0}\tcost: {1:>2.2f}'.format(i + 1, J / conf.log_period))
            final_cost = J / conf.log_period
            J = 0.0

        pos_g = 1.0 - sigmoid(score_pos)
        neg_g = sigmoid(score_neg)

        word_grad = -pos_g * context_vect + np.sum(as_matrix(neg_g) * negative_vects, axis=0)
        context_grad = -pos_g * word_vect
        neg_context_grad = as_matrix(neg_g) * as_matrix(word_vect).T

        Vp[word, :] -= learning_rate * word_grad
        Vo[context, :] -= learning_rate * context_grad
        Vo[neg_context, :] -= learning_rate * neg_context_grad

        if i % conf.decay_period == 0:
            learning_rate = learning_rate * conf.learning_rate_decay

    return Vp, Vo, final_cost

In [49]:
Vp, Vo, J = neg_sample(conf, train_set, train_tokens)

Update 10000	cost: 18.48
Update 20000	cost: 10.49
Update 30000	cost: 8.61
Update 40000	cost: 7.35
Update 50000	cost: 6.64
Update 60000	cost: 6.00
Update 70000	cost: 5.63
Update 80000	cost: 4.79
Update 90000	cost: 4.63
Update 100000	cost: 4.50
Update 110000	cost: 4.37
Update 120000	cost: 4.21
Update 130000	cost: 4.10
Update 140000	cost: 3.99
Update 150000	cost: 3.82
Update 160000	cost: 3.74
Update 170000	cost: 3.68
Update 180000	cost: 3.64
Update 190000	cost: 3.52
Update 200000	cost: 3.52
Update 210000	cost: 3.47
Update 220000	cost: 3.40
Update 230000	cost: 3.37
Update 240000	cost: 3.38
Update 250000	cost: 3.33
Update 260000	cost: 3.25
Update 270000	cost: 3.21
Update 280000	cost: 3.21
Update 290000	cost: 3.15
Update 300000	cost: 3.17
Update 310000	cost: 3.16
Update 320000	cost: 3.09
Update 330000	cost: 3.05
Update 340000	cost: 3.04
Update 350000	cost: 3.02
Update 360000	cost: 2.98
Update 370000	cost: 2.98
Update 380000	cost: 3.01
Update 390000	cost: 2.93
Update 400000	cost: 2.92
Update 

In [58]:
def lookup_word_idx(word, word_dict):
    try:
        return np.argwhere(np.array(word_dict) == word)[0][0]
    except:
        raise Exception("No such word in dict: {}".format(word))

def similar_words(embeddings, word, word_dict, hits):
    word_idx = lookup_word_idx(word, word_dict)
    similarity_scores = embeddings @ embeddings[word_idx]
    similar_word_idxs = np.argsort(-similarity_scores)    
    return [word_dict[i] for i in similar_word_idxs[:hits]]

In [69]:
import nltk
print('\n\nTraining cost: {0:>2.2f}\n\n'.format(J))

sample_words = [ 'good']

Vp_norm = Vp / as_matrix(np.linalg.norm(Vp , axis=1))
for w in sample_words:
    similar = similar_words(Vp_norm, w, train_dict, 5)
    print('Words similar to {}: {}'.format(w, ", ".join(similar)))

    #word_dic issue not fixed, not working properly



Training cost: 2.41


Words similar to good: good, unclog, purest, air, CONCORDE


In [52]:
#References
#[1] Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation.